### Import packages:

In [1]:
# Run below only if you are on Colab
!pip install chainer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import matplotlib.pyplot as plt

# If running on Colab, we need to specify the path of this notebook
import sys
if "google.colab" in sys.modules:
  # Set this to the directory of this file in Google Drive
  notebook_path = '/content/drive/MyDrive/CISC 455/light_weighted_gpu' 
  sys.path.append(notebook_path)
  from google.colab import files
  from google.colab import drive
  drive.mount('/content/drive')

from generator import fitness, generate
from mutation import gaussian_pertubation
from recombination import global_recombination
from tournament_selection import tournament_selection

latent_size = 60

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path = "."
model_dir = "saved_models"

if "google.colab" in sys.modules:
  file_path = notebook_path

## Experiment with EA:

### Hyperparameter:

In [4]:
MAX_ITERATION = 100

POP_SIZE = 100
WINNER_SIZE = 10
OFFSPRING_SIZE = 100 
MATING_POOL_SIZE = 30
TOURNAMENT_SIZE = 7

LEARNING_RATE = 0.1

# whether or not to use the n step size
n_step_size_mode = True

### Initialize population:
*   All entry of latent vectors are sampled from $[-3, 3]$.

In [5]:
population = []

for _ in range(POP_SIZE):
    latent_vec = 6 * np.random.sample(latent_size) - 3

    if n_step_size_mode:
        step_size = np.random.sample(latent_size)
    else:
        step_size = np.random.sample() * np.ones(latent_size)

    population.append([latent_vec, step_size])

### Fitness function:

In [6]:
def index_to_fitness(index):
    fitness_list = fitness(latent_vecs = [population[index][0].reshape(1,latent_size)], file_path = file_path, model_dir = model_dir)
    return fitness_list[0][4]

### Running EA:

In [7]:
for i in range(10):
    # Tournament selection
    winner_index, winner_fitness = tournament_selection(population, WINNER_SIZE, TOURNAMENT_SIZE, index_to_fitness)
    print(f"Iteration: {i+1}, max winner fitness: {max(winner_fitness)}, average winner fitness: {sum(winner_fitness) / WINNER_SIZE}")
    loser_index = []
    for j in range(len(population)):
        if j not in winner_index:
            loser_index.append(j)

    # Select parents
    select_from_losers = MATING_POOL_SIZE - WINNER_SIZE
    loser_parent_index = np.random.choice(loser_index, select_from_losers)
    mating_pool = []
    for j in range(len(population)):
        if (j in winner_index) or (j in loser_parent_index):
            mating_pool.append(population[j])

    # Recombination
    offspring = global_recombination(mating_pool, OFFSPRING_SIZE, n_step_size_mode)
    # Mutation
    offspring = gaussian_pertubation(offspring, LEARNING_RATE)

    # New population
    winners = []
    for j in range(len(population)):
        if j in winner_index:
            winners.append(population[j])
    
    winners = gaussian_pertubation(winners, LEARNING_RATE)

    selected_offspring_idx = np.random.choice(len(offspring), POP_SIZE - WINNER_SIZE)
    selected_offspring = [offspring[i] for i in selected_offspring_idx]

    population = []
    population.extend(winners)
    population.extend(selected_offspring)

print("Done")

Iteration: 1, max winner fitness: 114, average winner fitness: 65.0
Iteration: 2, max winner fitness: 115, average winner fitness: 77.7
Iteration: 3, max winner fitness: 115, average winner fitness: 86.2
Iteration: 4, max winner fitness: 115, average winner fitness: 89.2
Iteration: 5, max winner fitness: 114, average winner fitness: 65.5
Iteration: 6, max winner fitness: 115, average winner fitness: 77.6
Iteration: 7, max winner fitness: 81, average winner fitness: 62.8
Iteration: 8, max winner fitness: 108, average winner fitness: 81.4
Iteration: 9, max winner fitness: 131, average winner fitness: 82.0
Iteration: 10, max winner fitness: 131, average winner fitness: 82.8
Done


In [10]:
latent_vec_list = []
    
for winner in winners:
    mean = winner[0]
    var = winner[1]

    # Multi-variate normal
    latent_vec = np.random.normal(mean, var, size=(1, latent_size))

    latent_vec_list.append(latent_vec)
generate(latent_vecs = latent_vec_list, repeat_times = 1, file_path = file_path, model_dir = model_dir)


pig count: 3, tnt count: 0

pig count: 2, tnt count: 0

pig count: 2, tnt count: 1

pig count: 4, tnt count: 0

pig count: 1, tnt count: 0

pig count: 2, tnt count: 0

pig count: 1, tnt count: 0

pig count: 1, tnt count: 1

pig count: 3, tnt count: 0

pig count: 1, tnt count: 0
